In [ ]:
# Dataset Caltech 101
# Source: http://www.vision.caltech.edu/Image_Datasets/Caltech101/Caltech101.html
# Use 3 classes: elephant, rooster, rhino

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import join
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Find the name of each class
base_path = "/content/drive/My Drive/Data_FromCaltech"
class_names = listdir(base_path)
print("Num of classes:", len(class_names))

# Load all images from each class and label them. Use original size.
I = []
Y = np.empty(shape=(0))
for cl in range(len(class_names)):
    files = [join(base_path, class_names[cl], f) for f in listdir(join(base_path, class_names[cl]))]
    for file_path in files:
        I.append(imread(file_path))
    Y = np.concatenate((Y, cl * np.ones((len(files))) ))
    
print(len(I))
print(Y.shape)

Num of classes: 5
304
(304,)


In [ ]:
pip install opencv-python==3.4.2.16
pip install opencv-contrib-python==3.4.2.16
#Este codigo esta para evitar que cv2 nos marque error

SyntaxError: ignored

In [ ]:
# Compute SIFT for all images
sift_model = cv2.xfeatures2d.SIFT_create()
SIFTs = []
for img in I:
    _, s = sift_model.detectAndCompute(img, None)
    SIFTs.append(s)

print(SIFTs[0].shape)

error: ignored

In [ ]:
# Split data into training and validation sets
SIFTs_train, SIFTs_val, y_train, y_val = train_test_split(SIFTs, Y, test_size=0.2)

print(len(SIFTs_train))
print(len(SIFTs_val))
print(len(y_train))
print(len(y_val))

In [ ]:
# Count the total number of sift descriptors in the corpus.
num_sifts = 0
for s in SIFTs_train:
    num_sifts = num_sifts + len(s)
print(f"There are {num_sifts} SIFT descriptors in the training set.")

In [ ]:
# Prepare training set for clustering
trainingSIFTS = np.empty(shape=(0, 128))
for s in SIFTs_train:
    trainingSIFTS = np.concatenate((trainingSIFTS, s))

print(trainingSIFTS.shape)

In [ ]:
# Find visual vocabulary: Train a k-means clustering
n_words = 200
kmeans = KMeans(n_clusters=n_words).fit(trainingSIFTS) # This might take around 5 minutes for 70k SIFTs

In [ ]:
# Process all SIFTs from training set
X_train = np.zeros((len(SIFTs_train), n_words))
for ind, s in enumerate(SIFTs_train):
    words = kmeans.predict(s.astype(float))   # assign labels
    bow, _ = np.histogram(words, range(n_words + 1))
    X_train[ind] = bow / bow.sum()

print(X_train.shape)

In [ ]:
# Normalize data with a standar scaler
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [ ]:
# Process all SIFTs from validation set
X_val = np.zeros((len(SIFTs_val), n_words))
for ind, s in enumerate(SIFTs_val):
    words = kmeans.predict(s.astype(float))   # assign labels
    bow, _ = np.histogram(words, range(n_words + 1))
    X_val[ind] = bow / bow.sum()

print(X_val.shape)

In [ ]:
# Normalize data with a standar scaler
X_val = scaler.transform(X_val)

In [ ]:
# Classify with NaiveBayes
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
print(clf_nb.classes_)
print(clf_nb.class_count_)

# Print
print(f"Training mean accuracy: {clf_nb.score(X_train, y_train):6.4f}")
print(f"Test mean accuracy: {clf_nb.score(X_val, y_val):6.4f}")

In [ ]:
# Classify with SVM
clf_svm = SVC(kernel='linear')
clf_svm.fit(X_train, y_train)
print(clf_svm.classes_)

# Print
print(f"Training mean accuracy: {clf_svm.score(X_train, y_train):6.4f}")
print(f"Test mean accuracy: {clf_svm.score(X_val, y_val):6.4f}")

# Mini proyecto Bow and SVM

In [ ]:
# Usando los parámetros default de Naive Bayes y de SVC (excepto el kernel. usen lineal), encuentren el tamaño de vocabulario
# que proporcione el mejor promedio de exactitud de clasificación (en entrenamiento y test, traten de evitar sobre entrenamiento).
# Prueba los siguientes valores: 50, 100, 250, 500, 1000, 2000 palabras visuales.
# Nota, para 2000 palabras, kmeans puede tomar al rededor de 30 min, mientras que para 50 palabras tomará únicamente unos
# 2 minutos. Puedes usar el parámetro verbose para controlar si quieres información sobre lo que kmeans está haciendo.

# == Q1 ==
# Reporta una tabla con los scores de clasificación para entrenamiento y validación para ambos modelos y los 6 tamaños de
# diccionario visual.

val_palabras = [50, 100, 250, 500, 1000, 2000]
lista_val = []
for val in val_palabras:
  SIFTs_train, SIFTs_val, y_train, y_val = train_test_split(SIFTs, Y, test_size=0.2)

  num_sifts = 0
  for s in SIFTs_train:
    num_sifts = num_sifts + len(s)

  trainingSIFTS = np.empty(shape=(0, 128))
  for s in SIFTs_train:
    trainingSIFTS = np.concatenate((trainingSIFTS, s))

  kmeans = KMeans(n_clusters=val).fit(trainingSIFTS) 

  X_train = np.zeros((len(SIFTs_train), val))
  for ind, s in enumerate(SIFTs_train):
    words = kmeans.predict(s.astype(float))   # assign labels
    bow, _ = np.histogram(words, range(val + 1))
    X_train[ind] = bow / bow.sum()

  scaler = MinMaxScaler().fit(X_train)
  X_train = scaler.transform(X_train)

  X_val = np.zeros((len(SIFTs_val), val))
  for ind, s in enumerate(SIFTs_val):
    words = kmeans.predict(s.astype(float))   # assign labels
    bow, _ = np.histogram(words, range(val + 1))
    X_val[ind] = bow / bow.sum()

  X_val = scaler.transform(X_val)

  # Classify with NaiveBayes
  clf_nb = MultinomialNB()
  clf_nb.fit(X_train, y_train)
  #print(clf_nb.classes_)
  #print(clf_nb.class_count_)

  # Print
  #print(f"Training mean accuracy BAYES: {clf_nb.score(X_train, y_train):6.4f}", " CON VAL ", val)
  #print(f"Test mean accuracy BAYES: {clf_nb.score(X_train, y_train):6.4f}", " CON VAL ", val)

  # Classify with SVM
  clf_svm = SVC(kernel='linear')
  clf_svm.fit(X_train, y_train)
  #print(clf_svm.classes_)

  # Print
  #print(f"Training mean accuracy SVM: {clf_svm.score(X_train, y_train):6.4f}", " CON VAL ", val)
  #print(f"Test mean accuracy SVM: {clf_svm.score(X_val, y_val):6.4f}", " CON VAL ", val)
  lista_val.append([val, clf_nb.score(X_train, y_train), clf_nb.score(X_train, y_train), clf_svm.score(X_train, y_train), clf_svm.score(X_val, y_val)])

In [ ]:
#Impresión de valores para pregunta 1
import pandas as pd
dataf = pd.DataFrame(lista_val, columns=["Número de palabras", "Training mean accuracy BAYES", "Test mean accuracy BAYES", "Training mean accuracy SVM", "Test mean accuracy SVM"])
dataf

In [ ]:
# Ahora congelen el tamaño del vocabulario, usando el mejor obtenido anteriormente.
# Con Naive Bayes no hay mucho más que hacer, pero podemos explorar los hiperparámetros de SVM.
# Varien el tipo de kernel para SVC, probando linear, rbf y polinimial. Y varien los parámetros C y gamma.
# Pueden evaluar: 0.01, 0.05, 0.1, 0.5, 1, 5 para ambos parámetros
# Encuentren la combinación con el mejor desempeño de clasificación (en entrenamiento y test, eviten sobre entrenamiento).
# Revisen la documentación de GridSearch en sklearn, puede ayudarles a eficientar esta exploración.

val = 100

SIFTs_train, SIFTs_val, y_train, y_val = train_test_split(SIFTs, Y, test_size=0.2)

num_sifts = 0
for s in SIFTs_train:
  num_sifts = num_sifts + len(s)

trainingSIFTS = np.empty(shape=(0, 128))
for s in SIFTs_train:
  trainingSIFTS = np.concatenate((trainingSIFTS, s))

kmeans = KMeans(n_clusters=val).fit(trainingSIFTS) 

X_train = np.zeros((len(SIFTs_train), val))
for ind, s in enumerate(SIFTs_train):
  words = kmeans.predict(s.astype(float))   # assign labels
  bow, _ = np.histogram(words, range(val + 1))
  X_train[ind] = bow / bow.sum()

scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)

X_val = np.zeros((len(SIFTs_val), val))
for ind, s in enumerate(SIFTs_val):
  words = kmeans.predict(s.astype(float))   # assign labels
  bow, _ = np.histogram(words, range(val + 1))
  X_val[ind] = bow / bow.sum()

X_val = scaler.transform(X_val)

from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.01, 0.05, 0.1, 0.5, 1, 5], 'gamma':[0.01, 0.05, 0.1, 0.5, 1, 5]}

clf_svm = SVC()
clf_GS = GridSearchCV(clf_svm, parameters)
clf_GS.fit(X_train, y_train)


#print(f"Training mean accuracy SVM: {clf_GS.score(X_train, y_train):6.4f}", " CON VAL ", val)
#print(f"Test mean accuracy SVM: {clf_GS.score(X_val, y_val):6.4f}", " CON VAL ", val)
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


In [ ]:
# == Q2 ==
# Para cada tipo de kernel, reporten una tabla con las combinaciones de C y gamma evaluadas y los desempeños (train, val)
# obtenidos.

res = clf_GS.cv_results_
df = pd.DataFrame(data=res)
df

In [ ]:
#Pregunta 2 separada por kernel reflejando los valores de cada iteración:

kernel_S=['linear', 'rbf', 'poly']
C_S=[0.01, 0.05, 0.1, 0.5, 1, 5]
gamma_S=[0.01, 0.05, 0.1, 0.5, 1, 5]
dic = {}
i = 1
for ker in kernel_S:
  for ce in C_S:
    for gam in gamma_S:

      val = 100

      SIFTs_train, SIFTs_val, y_train, y_val = train_test_split(SIFTs, Y, test_size=0.2)

      num_sifts = 0
      for s in SIFTs_train:
        num_sifts = num_sifts + len(s)

      trainingSIFTS = np.empty(shape=(0, 128))
      for s in SIFTs_train:
        trainingSIFTS = np.concatenate((trainingSIFTS, s))

      kmeans = KMeans(n_clusters=val).fit(trainingSIFTS) 

      X_train = np.zeros((len(SIFTs_train), val))
      for ind, s in enumerate(SIFTs_train):
        words = kmeans.predict(s.astype(float))   # assign labels
        bow, _ = np.histogram(words, range(val + 1))
        X_train[ind] = bow / bow.sum()

      scaler = MinMaxScaler().fit(X_train)
      X_train = scaler.transform(X_train)

      X_val = np.zeros((len(SIFTs_val), val))
      for ind, s in enumerate(SIFTs_val):
        words = kmeans.predict(s.astype(float))   # assign labels
        bow, _ = np.histogram(words, range(val + 1))
        X_val[ind] = bow / bow.sum()

      X_val = scaler.transform(X_val)

      clf_svm = SVC(kernel=ker,gamma=gam, C=ce)
      clf_svm.fit(X_train, y_train)
      print(clf_svm.classes_)

      # Print
      print(f"Training mean accuracy SVM: {clf_svm.score(X_train, y_train):6.4f}", " CON Gamma ",gam, ", C ",ce,", kernel ", ker)
      dic[i] = [{clf_svm.score(X_train, y_train):6.4f}, gam, ce, ker]
      i += 1
      print(f"Test mean accuracy SVM: {clf_svm.score(X_val, y_val):6.4f}", " CON Gamma ",gam, ", C ",ce,", kernel ", ker)
      dic[i] = = [{clf_svm.score(X_val, y_val):6.4f}, gam, ce, ker]
      i += 1

In [ ]:
# == Q3 ==
# De entre todos los modelos evaluados, elijan el que consideren mejor. Reporten su justificación para considerarlo el mejor.

# De todos los modelos evaluados, que incluyeron kernel lineal, RBF y polinomial consideramos que el que presentó los
# resultados más satisfactorios fue el polinomial. Este con un valor C = 0.01 y gamma = 0.5. Esto nos dió un score que imprimimos
# que imprimimos en esta celda. Este modelo es mejor dado a que se tiene una alta media en ambos casos, 
# demostrando que no se tiene caso de overfitting ni underfitting. Con estos parámetros se alcanzó una media de prueba más 
# alta que la mayoría de los otros modelos, en los cuales el score fue menor. Otras pruebas demostraron
# un valor más alto en media de entrenamiento, pero ninguno tuvo una media de prueba igual o más alto que este. Y revisando
# para RBF y polinomial, los peores casos eran mucho menores que los resultados vistos. El score toma las medias de 
# entrenamiento y prueba para sacar el mejor modelo, por lo que buscamos el que tenga mayor puntaje en esto. Así aseguramos
# la mayor precisión.

print("La mejor score es:")
print(clf_GS.best_score_)

print("Los parámetros que alcanzaron esta score fueron: ")
print(clf_GS.best_params_)

In [ ]:
# == Q4 ==
# Usando el mejor modelo de entre todos los probados, grafiquen y reporten la matriz de confusión.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

mod_svm = SVC(kernel='poly',C=0.01, gamma=0.5)
modelo = mod_svm.fit(X_train, y_train)

#La matriz de confusión no se encuentra normalizada, por lo cual los valores no van de 0 a 1
disp = plot_confusion_matrix(modelo, X_val, y_val, cmap=plt.cm.Blues)
disp.ax_.set_title("Matriz de confusión sin normalizar")
print("Matriz de confusión sin normalizar:")
print(disp.confusion_matrix)

In [ ]:
# == Q5 ==
# Con base en el mejor modelo (seleccionado)
# ¿Qué clase es la más fácil de clasificar? ¿Por qué creen que sea así?

# Rooster o gallo es la más fácil de clasificar. Esto porque se necesitan menos palabras para asociar una imagen a 
# este concepto. Al ser menos compleja de escribir, también se eliminan varias búsquedas que involucran verificar errores 
# del usuario. Por lo tanto, el clasificador tomaría menos tiempo para relacionar las imágenes. Además que este animal tiene 
# una forma reconocible para el algoritmo SIFT. El kernel RBF, al tomar el radio alrededor de un punto dado, puede rápidamente 
# que los valores son menores que en los otros casos. Por lo que descarta opciones que no son relevantes a la búsqueda con mayor
# eficiencia.

# ¿Cuáles clases son las que más se confunden entre sí? ¿Por qué creen que sea así?

# Rhino y elephant son más confundidas entre sí. Esto dado que ambos tienen características similares, que al momento de 
# ejecutar el algoritmo SIFT pueden ser difíciles de discernir. En cuanto forma, estas dos clases son más similares entre ellas
# comparando con rooster, por lo cual el radio alrededor de los puntos clave es parecido en ambos casos. 
# Por lo que es necesario dar más tiempo para notar los detalles en cada imagen y categorizar correctamente cada una de estas. 

In [ ]:
# Bonus: Extender la exploración de la pregunta 2, para evaluar:
# 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50
# ¿Se logró alguna mejora?
val = 100

SIFTs_train, SIFTs_val, y_train, y_val = train_test_split(SIFTs, Y, test_size=0.2)

num_sifts = 0
for s in SIFTs_train:
  num_sifts = num_sifts + len(s)

trainingSIFTS = np.empty(shape=(0, 128))
for s in SIFTs_train:
  trainingSIFTS = np.concatenate((trainingSIFTS, s))

kmeans = KMeans(n_clusters=val).fit(trainingSIFTS) 

X_train = np.zeros((len(SIFTs_train), val))
for ind, s in enumerate(SIFTs_train):
  words = kmeans.predict(s.astype(float))   # assign labels
  bow, _ = np.histogram(words, range(val + 1))
  X_train[ind] = bow / bow.sum()

scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)

X_val = np.zeros((len(SIFTs_val), val))
for ind, s in enumerate(SIFTs_val):
  words = kmeans.predict(s.astype(float))   # assign labels
  bow, _ = np.histogram(words, range(val + 1))
  X_val[ind] = bow / bow.sum()

X_val = scaler.transform(X_val)

from sklearn.model_selection import GridSearchCV

parameters_b = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50], 'gamma':[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]}

clf_svm_b = SVC()
clf_GS_b = GridSearchCV(clf_svm_b, parameters_b)
clf_GS_b.fit(X_train, y_train)

In [ ]:
print("La mejor score con los valores adicionales es:")
print(clf_GS_b.best_score_)

print("Los parámetros que alcanzaron esta score fueron: ")
print(clf_GS_b.best_params_)

In [ ]:
# ¿Se logró alguna mejora?
# Si, pero varía dependiendo las iteraciones.
# En un primer intento la clasificación se realizó con la misma eficiencia, a pesar de agregar nuevos parámetros a revisar. 
# Y con estos nuevos valores se alcanzó una score más alta que la anterior, aunque esta diferencia no sea tan grande como 
# se quisiera. No obstante, con esto podemos identificar un kernel más eficiente, en este caso RBF, 
# usando un valor C = 10 que no se tenía en la prueba anterior.  
# Sin embargo, al reiniciar el kernel y correr el proceso una segunda vez, se obtuvo un resultado distinto. El score fue más
# alto, alcanzando un 0.62, mientras que la primera tuvo un valor aproximado de 0.59 (cerca del 0.6 pero no lo alcanzaba).
# El caso en esta iteración, es que tanto los parámetros C como gamma se encontraban dentro de las posibilidades que 
# se establecieron en la pregunta 2. Así que esto puede ser debido a eficiencia al realizar la prueba o recursos disponibles
# del ordenador.

# Por lo tanto, en ambas pruebas encontramos una mejor score utilizando un rango mayor de parámetros. A pesar de que los valores
# ideales se encontraban en ambas listas definidas, en algunos casos las mejoras provienen de un uso de C o gamma presente
# únicamente en esta extensión de la exploración.

In [ ]:
# Entrega. PDF, con nombres. Se vale que sea la impresión de las salidas del notebook, pero excluyan la parte de muestra.
# Incluyan sólo lo que corresponde a responder las preguntas.
# Deadline: Lunes 5 de octubre, 18:00 hr.